In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,463 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-sec

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-21 16:51:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-21 16:51:20 (6.57 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NewsPopularity").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://onlinenews-1.s3.us-west-1.amazonaws.com/OnlineNewsPopularity_clean.csv"
spark.sparkContext.addFile(url)
news_df = spark.read.csv(SparkFiles.get("OnlineNewsPopularity_clean.csv"), sep=",", header=True, inferSchema=True)

In [5]:
news_df.show()

+--------------------+---------+--------------+----------------+---------------+----------------+------------------------+---------+--------------+--------+----------+--------------------+------------+-------------------------+-----------------------------+-------------------+----------------------+--------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------------+-------------------------+--------------------------+-----------------+------------------+--------------------+-------------------+-----------------+-------------------+-----------------+----------+-----------+-----------+-----------+-----------+-----------+-------------------+-------------------------+--------------------------+--------------------------+-------------------+-------------------+---------------------+----------------------+----------------------+---------------------+----------------------+---------------------

In [6]:
time_df = news_df.select(["url","weekday_is_monday","weekday_is_tuesday","weekday_is_wednesday", "weekday_is_thursday","weekday_is_friday", "weekday_is_saturday", "weekday_is_sunday","is_weekend"])
time_df.show()

+--------------------+-----------------+------------------+--------------------+-------------------+-----------------+-------------------+-----------------+----------+
|                 url|weekday_is_monday|weekday_is_tuesday|weekday_is_wednesday|weekday_is_thursday|weekday_is_friday|weekday_is_saturday|weekday_is_sunday|is_weekend|
+--------------------+-----------------+------------------+--------------------+-------------------+-----------------+-------------------+-----------------+----------+
|http://mashable.c...|                1|                 0|                   0|                  0|                0|                  0|                0|         0|
|http://mashable.c...|                1|                 0|                   0|                  0|                0|                  0|                0|         0|
|http://mashable.c...|                1|                 0|                   0|                  0|                0|                  0|                0|    

In [7]:
shares_df = news_df.select(["url", "shares"])
shares_df.show()

+--------------------+------+
|                 url|shares|
+--------------------+------+
|http://mashable.c...|   593|
|http://mashable.c...|   711|
|http://mashable.c...|  1500|
|http://mashable.c...|  1200|
|http://mashable.c...|   505|
|http://mashable.c...|   855|
|http://mashable.c...|   556|
|http://mashable.c...|   891|
|http://mashable.c...|  3600|
|http://mashable.c...|   710|
|http://mashable.c...|  2200|
|http://mashable.c...|  1900|
|http://mashable.c...|   823|
|http://mashable.c...| 10000|
|http://mashable.c...|   761|
|http://mashable.c...|  1600|
|http://mashable.c...| 13600|
|http://mashable.c...|  3100|
|http://mashable.c...|  5700|
|http://mashable.c...| 17100|
+--------------------+------+
only showing top 20 rows



In [8]:
category_df = news_df.select(["url", "data_channel_is_lifestyle", "data_channel_is_entertainment", "data_channel_is_bus", "data_channel_is_socmed", "data_channel_is_tech", "data_channel_is_world"])
category_df.show()

+--------------------+-------------------------+-----------------------------+-------------------+----------------------+--------------------+---------------------+
|                 url|data_channel_is_lifestyle|data_channel_is_entertainment|data_channel_is_bus|data_channel_is_socmed|data_channel_is_tech|data_channel_is_world|
+--------------------+-------------------------+-----------------------------+-------------------+----------------------+--------------------+---------------------+
|http://mashable.c...|                        0|                            1|                  0|                     0|                   0|                    0|
|http://mashable.c...|                        0|                            0|                  1|                     0|                   0|                    0|
|http://mashable.c...|                        0|                            0|                  1|                     0|                   0|                    0|
|http://ma

In [9]:
words_df = news_df.select(["url", "n_tokens_content", "n_tokens_title", "n_unique_tokens", "n_non_stop_words", "n_non_stop_unique_tokens", "average_token_length"])
words_df.show()

+--------------------+----------------+--------------+---------------+----------------+------------------------+--------------------+
|                 url|n_tokens_content|n_tokens_title|n_unique_tokens|n_non_stop_words|n_non_stop_unique_tokens|average_token_length|
+--------------------+----------------+--------------+---------------+----------------+------------------------+--------------------+
|http://mashable.c...|             219|            12|    0.663594467|     0.999999992|             0.815384609|         4.680365297|
|http://mashable.c...|             255|             9|    0.604743081|     0.999999993|             0.791946303|          4.91372549|
|http://mashable.c...|             211|             9|    0.575129531|     0.999999992|             0.663865541|         4.393364929|
|http://mashable.c...|             531|             9|    0.503787878|     0.999999997|             0.665634673|         4.404896422|
|http://mashable.c...|            1072|            13|    0.41

In [10]:
processing_df = news_df.select(["url", "LDA_00", "LDA_01", "LDA_02", "LDA_03", "LDA_04"])
processing_df.show()

+--------------------+-----------+-----------+-----------+-----------+-----------+
|                 url|     LDA_00|     LDA_01|     LDA_02|     LDA_03|     LDA_04|
+--------------------+-----------+-----------+-----------+-----------+-----------+
|http://mashable.c...|0.500331204| 0.37827893|0.040004675|0.041262648|0.040122544|
|http://mashable.c...|0.799755687|0.050046675|0.050096252|0.050100673|0.050000712|
|http://mashable.c...|0.217792289|0.033334457|0.033351425|0.033333536|0.682188294|
|http://mashable.c...|0.028573216|0.419299642|0.494650826|0.028904718|0.028571598|
|http://mashable.c...| 0.02863281|0.028793552|0.028575185|0.028571675|0.885426778|
|http://mashable.c...|0.022245276|0.306717576|0.022231278| 0.02222429|0.626581581|
|http://mashable.c...|0.020081666|0.114705387|0.020024369|0.020015328| 0.82517325|
|http://mashable.c...|0.022224357|0.150732973|0.243435476|0.022223603|0.561383591|
|http://mashable.c...|0.458250415|0.028979431|0.028661883|0.029695859|0.454412412|
|htt

In [11]:
keywords_df = news_df.select(["url", "kw_min_min", "kw_max_min", "kw_avg_min", "kw_min_max", "kw_max_max", "kw_avg_max", "kw_min_avg", "kw_max_avg", "kw_avg_avg"])
keywords_df.show()

+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|                 url|kw_min_min|kw_max_min|kw_avg_min|kw_min_max|kw_max_max|kw_avg_max|kw_min_avg|kw_max_avg|kw_avg_avg|
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|http://mashable.c...|         0|       0.0|       0.0|         0|         0|       0.0|       0.0|       0.0|       0.0|
|http://mashable.c...|         0|       0.0|       0.0|         0|         0|       0.0|       0.0|       0.0|       0.0|
|http://mashable.c...|         0|       0.0|       0.0|         0|         0|       0.0|       0.0|       0.0|       0.0|
|http://mashable.c...|         0|       0.0|       0.0|         0|         0|       0.0|       0.0|       0.0|       0.0|
|http://mashable.c...|         0|       0.0|       0.0|         0|         0|       0.0|       0.0|       0.0|       0.0|
|http://mashable.c...|  

In [12]:
links_df = news_df.select(["url", "num_hrefs", "num_self_hrefs", "self_reference_min_shares", "self_reference_max_shares", "self_reference_avg_sharess"])
links_df.show()

+--------------------+---------+--------------+-------------------------+-------------------------+--------------------------+
|                 url|num_hrefs|num_self_hrefs|self_reference_min_shares|self_reference_max_shares|self_reference_avg_sharess|
+--------------------+---------+--------------+-------------------------+-------------------------+--------------------------+
|http://mashable.c...|        4|             2|                    496.0|                    496.0|                     496.0|
|http://mashable.c...|        3|             1|                      0.0|                      0.0|                       0.0|
|http://mashable.c...|        3|             1|                    918.0|                    918.0|                     918.0|
|http://mashable.c...|        9|             0|                      0.0|                      0.0|                       0.0|
|http://mashable.c...|       19|            19|                    545.0|                  16000.0|            

In [13]:
media_df = news_df.select(["url", "num_imgs", "num_videos"])
media_df.show()

+--------------------+--------+----------+
|                 url|num_imgs|num_videos|
+--------------------+--------+----------+
|http://mashable.c...|       1|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|      20|         0|
|http://mashable.c...|       0|         0|
|http://mashable.c...|      20|         0|
|http://mashable.c...|      20|         0|
|http://mashable.c...|       0|         0|
|http://mashable.c...|       1|         1|
|http://mashable.c...|       1|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|      11|         0|
|http://mashable.c...|       0|        21|
|http://mashable.c...|       9|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|      12|         1|
|http://mashable.c...|       1|         0|
|http://mashable.c...|       1|         0|
|http://mashable.c...|      42|         0|
+----------

In [14]:
polarity_df = news_df.select(["url","timedelta", "rate_positive_words", "rate_negative_words", "avg_positive_polarity","avg_negative_polarity", "title_subjectivity", "title_sentiment_polarity", "abs_title_subjectivity","abs_title_sentiment_polarity"])
polarity_df.show()

+--------------------+---------+-------------------+-------------------+---------------------+---------------------+------------------+------------------------+----------------------+----------------------------+
|                 url|timedelta|rate_positive_words|rate_negative_words|avg_positive_polarity|avg_negative_polarity|title_subjectivity|title_sentiment_polarity|abs_title_subjectivity|abs_title_sentiment_polarity|
+--------------------+---------+-------------------+-------------------+---------------------+---------------------+------------------+------------------------+----------------------+----------------------------+
|http://mashable.c...|      731|        0.769230769|        0.230769231|          0.378636364|                -0.35|               0.5|                 -0.1875|                   0.0|                      0.1875|
|http://mashable.c...|      731|        0.733333333|        0.266666667|          0.286914601|             -0.11875|               0.0|             

In [16]:
shares_v_polarity_df = shares_df.join(polarity_df,on='url').distinct()
shares_v_polarity_df.show()

+--------------------+------+---------+-------------------+-------------------+---------------------+---------------------+------------------+------------------------+----------------------+----------------------------+
|                 url|shares|timedelta|rate_positive_words|rate_negative_words|avg_positive_polarity|avg_negative_polarity|title_subjectivity|title_sentiment_polarity|abs_title_subjectivity|abs_title_sentiment_polarity|
+--------------------+------+---------+-------------------+-------------------+---------------------+---------------------+------------------+------------------------+----------------------+----------------------------+
|http://mashable.c...|  1900|      727|                0.5|                0.5|          0.320833333|         -0.307589286|               0.0|                     0.0|                   0.5|                         0.0|
|http://mashable.c...|  1100|      720|                0.8|                0.2|          0.325252525|          -0.207870

In [18]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://news-data.cru88xwlpqpl.us-west-1.rds.amazonaws.com:5432/news_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [17]:
time_df.write.jdbc(url=jdbc_url, table='time', mode=mode, properties=config)

In [19]:
shares_df.write.jdbc(url=jdbc_url, table='shares', mode=mode, properties=config)

In [ ]:
category_df.write.jdbc(url=jdbc_url, table='category', mode=mode, properties=config)

In [ ]:
words_df.write.jdbc(url=jdbc_url, table='words', mode=mode, properties=config)

In [ ]:
processing_df.write.jdbc(url=jdbc_url, table='processing', mode=mode, properties=config)

In [ ]:
keywords_df.write.jdbc(url=jdbc_url, table='keywords', mode=mode, properties=config)

In [ ]:
links_df.write.jdbc(url=jdbc_url, table='links', mode=mode, properties=config)

In [ ]:
media_df.write.jdbc(url=jdbc_url, table='media', mode=mode, properties=config)

In [20]:
polarity_df.write.jdbc(url=jdbc_url, table='polarity', mode=mode, properties=config)

In [21]:
shares_v_polarity_df.write.jdbc(url=jdbc_url, table='shares_v_polarity', mode=mode, properties=config)